<a href="https://colab.research.google.com/github/dongjaeseo/colab/blob/main/backpropagation3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
train_data = np.loadtxt('/content/drive/MyDrive/data/backprop/training.dat', unpack = True)
test_data = np.loadtxt('/content/drive/MyDrive/data/backprop/testing.dat', unpack = True)

In [ ]:
train_label = []
test_label = []
for i in range(3):
    for j in range(25):
        label = np.zeros(3)
        label[i] = 1
        train_label.append(label)
        test_label.append(label)
    
train_label = np.array(train_label).T
test_label = np.array(test_label).T

In [ ]:
# Define Activation Functions
def sigmoid(x):
    return 1/(1+np.exp(-x))

def relu(x):
    return np.maximum(0,x)

def softmax(x):
    return np.exp(x)/np.sum(np.exp(x),axis=0)

In [ ]:
nx, nh1, nh2, ny = 4, 3, 3, 3
U = np.random.randn(nx , nh1)*2
V = np.random.randn(nh1, nh2)*2
W = np.random.randn(nh2, ny)*2
learning_rate = 1e-1

In [ ]:
input = np.zeros(nx)

h1_out, h1_deriv = np.zeros(nh1), np.zeros(nh1) # 순전파시 계산 - 은닉계층 1
h1_delta = np.zeros(nh1)                        # 역전파시 계산

h2_out, h2_deriv = np.zeros(nh2), np.zeros(nh2) # 순전파시 계산 - 은닉계층 2
h2_delta = np.zeros(nh2)                        # 역전파시 계산

y_out, y_deriv = np.zeros(ny), np.zeros(ny)     # 순전파시 계산 - 출력계층
y_delta = np.zeros(ny)                          # 역전파시 계산

In [ ]:
def model(X, Y, learning_rate = 0.1, num_iterations = 20, activation = sigmoid):
    layer_dims = [X.shape[0], 4, 4, 3]  
    grads = {}
    train_costs = []
    test_costs = []

    # Initialize weights - Glorot Initialization
    parameters = {}
    for i in range(1, len(layer_dims)):
        m = np.sqrt(6/(layer_dims[i]+layer_dims[i-1]))
        parameters['W' + str(i)] = np.random.uniform(-m, m, size = (layer_dims[i], layer_dims[i-1]))
    
    for i in range(0, num_iterations):       
        # Forward propagation
        W1 = parameters["W1"]
        W2 = parameters["W2"]
        W3 = parameters["W3"]

        # Forward propagation
        Z1 = np.dot(W1, X)
        A1 = activation(Z1)
        Z2 = np.dot(W2, A1)
        A2 = activation(Z2)
        Z3 = np.dot(W3, A2)
        Y_pred = softmax(Z3)

        # Cost calculation (Cross Entropy)
        m = Y.shape[1]
    
        logprobs = np.multiply(-np.log(Y_pred),Y) + np.multiply(-np.log(1 - Y_pred), 1 - Y)
        cost = 1./m * np.nansum(logprobs)  

        # Backward propagation
        m = X.shape[1]
        
        dZ3 = Y_pred - Y
        dW3 = 1./m * np.dot(dZ3, A2.T)
        
        dA2 = np.dot(W3.T, dZ3)
        dZ2 = np.multiply(dA2, np.int64(A2 > 0))
        dW2 = 1./m * np.dot(dZ2, A1.T)
        
        dA1 = np.dot(W2.T, dZ2)
        dZ1 = np.multiply(dA1, np.int64(A1 > 0))
        dW1 = 1./m * np.dot(dZ1, X.T)
        
        grads = {"dZ3": dZ3, "dW3": dW3, 
                 "dA2": dA2, "dZ2": dZ2, "dW2": dW2, 
                 "dA1": dA1, "dZ1": dZ1, "dW1": dW1,}

        # Weight Updation
        for k in range(len(parameters)//2):
            parameters["W" + str(k+1)] = parameters["W" + str(k+1)] - learning_rate * grads["dW" + str(k+1)]
            
        print("Cost after iteration {}: {}".format(i, cost))
        train_costs.append(cost)
    
    # Plot the cost
    plt.plot(train_costs, color = 'blue')
    plt.ylabel('training_cost')
    plt.xlabel('iterations')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()

    return parameters

In [ ]:
parameters = model(train_data, train_label, learning_rate = 0.1, num_iterations = 200, activation = relu)

In [ ]:
# Predict Function
def predict(X, Y, parameters, activation = relu):
    W1 = parameters["W1"]
    W2 = parameters["W2"]
    W3 = parameters["W3"]

    Z1 = np.dot(W1, X)
    A1 = activation(Z1)
    Z2 = np.dot(W2, A1)
    A2 = activation(Z2)
    Z3 = np.dot(W3, A2)
    Y_pred = softmax(Z3)

    prediction = np.argmax(Y_pred, axis = 0)
    return prediction

# Calculate Training and Test Accuracy
train_pred = predict(train_data, train_label, parameters)
print ("On the training set:")
print("Accuracy: "  + str(np.mean((train_pred[:] == np.argmax(train_label, axis = 0)))))

test_pred = predict(test_data, test_label, parameters)
print ("On the test set:")
print("Accuracy: "  + str(np.mean((test_pred[:] == np.argmax(test_label, axis = 0)))))

In [ ]:
from sklearn.metrics import confusion_matrix
import pandas as pd
import seaborn as sns

cm = confusion_matrix(np.argmax(train_label, axis = 0), test_pred)
cm_df = pd.DataFrame(cm)

plt.figure(figsize=(5,4))
sns.heatmap(cm_df, annot=True)
plt.title('Confusion Matrix_tune')
plt.ylabel('Actal Values')
plt.xlabel('Predicted Values')
plt.show()